In [1]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
from scipy.optimize import minimize

## Group Assignment
### Team Number: 11
### Team Member Names: Akram, Annie, Jester
### Team Strategy Chosen: Market Beat

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [2]:
# Function to get the tickers from the provided CSV file. 
def get_tickers():
    tickers = pd.read_csv('Tickers.csv')
    ticker_lst = [tickers.columns[0]] + (list(tickers[tickers.columns[0]]))
    return ticker_lst

In [3]:
# Important Constants: 
amount = 1_000_000 # Initial investment amount of $1,000,000
group = 11
start_date, end_date = '2022-09-30', '2024-09-30' # Start and end date of the simulation
min_stocks, max_stocks = 12, 24
# Reading in CSV file: 
ticker_lst = get_tickers()

# Initializing variable to store the tickers we will use in our portfolio
columns = ['Ticker', 'Price', 'Currency', 'Shares', 'Value', 'Weight']
Portfolio_Final = pd.DataFrame(columns=columns)

In [4]:
exchange_rate = yf.Ticker('CAD=X').fast_info['last_price']
print(f'The current exchange rate for the latest available day:\nUSD -> CAD: ${np.round(exchange_rate, 4)}')

The current exchange rate for the latest available day:
USD -> CAD: $1.4019


In [5]:
# Filtering valid stocks by inputting a list of strings for each ticker. 
def filter_stocks(ticker_lst):
    valid_tickers, invalid_tickers = {}, []
    # Loop through all tickers to check if they are valid
    for ticker in ticker_lst:
        stock = yf.Ticker(ticker)
        try:
            info = stock.fast_info # Get basic stock info

            hist = stock.history(start=start_date, end=end_date) # Get stock history
            hist.index = hist.index.strftime('%Y-%m-%d')

            avg_volume = hist.loc[((hist.index >= '2023-09-30') & (hist.index <= '2024-09-30'))]['Volume'].mean() # Calculate average volume in specified date range.
            currency = info.get("currency")
            if ((hist.empty is not None) and # filter for stocks delisted on yfinance
                ( currency == "USD" or currency == "CAD") and # filter for stocks that are not USD
                (avg_volume >= 100_000)): # Filter by volume greater than 100,000
                if currency == "CAD":
                    valid_tickers[ticker] = hist['Close'] # Store the close prices of the stock as a Series
                elif currency == "USD":
                    valid_tickers[ticker] = hist['Close'] * exchange_rate # Convert USD to CAD
            else:
                invalid_tickers.append(ticker)
        except:
            invalid_tickers.append(ticker)
    return [valid_tickers, invalid_tickers]
    # valid_tickers is a dictionary of Series where the key is the name of the ticker. 
    # invalid_tickers is a list of ticker strings which were removed in the filtering process. 

In [6]:
# sharpe ratio optimization

def optimal_sharpe(tickers, start_date, end_date, risk_free_rate, investment):
    # download data
    data = yf.download(tickers, start=start_date, end=end_date)['Close']

    # calculate mean return of stocks and covariance of stocks
    returns = data.pct_change()
    returns.drop(index=returns.index[0], inplace = True)
    mean_returns = returns.mean()
    covariance_matrix = returns.cov()

    def neg_sharpe(weights):
        #alternate
        #portfolio = data/data.iloc[0] # normalize returns
        #portfolio = portfolio*weights*investment
        #portfolio['total'] = portfolio.sum(axis=1)
        #portfolio['daily return'] = portfolio['total'].pct_change(1)

        #er = portfolio['daily return'].mean()
        #std = portfolio['daily return'].std()
        #sr = er/std
        


        # calculate portfolio expected return by weighing each stock's expected return
        num_days = len(returns)
        portfolio_expected_return = np.sum(weights*mean_returns*num_days)

        portfolio_variance = 0
        # calculate portfolio risk (std) by finding the portfolio variance, which is affected by covariance
        for i in range(len(weights)):
            for j in range(len(weights)):
                portfolio_variance += weights[i] * weights[j] * covariance_matrix.iloc[i, j]*num_days
        portfolio_std = np.sqrt(portfolio_variance)
        
        # calculate sharpe ratio
        sharpe = (portfolio_expected_return - risk_free_rate)/portfolio_std
        
        return -sharpe #make sharpe ratio negative for minimize function

    # constraints
    def check_sum(weights): 
        return np.sum(weights)-1 #returns 0 if weights sum up to 1
    constraints = {'type': 'eq', 'fun': check_sum}

    min_weight = 1/(2*len(tickers))
    max_weight = 0.4

    bounds = [(min_weight, max_weight)]*len(tickers)

    # initial guess
    init_guess = [1.0/len(tickers)]*len(tickers)

    results = minimize(neg_sharpe, init_guess, method="SLSQP", bounds=bounds, constraints=constraints)

    return results


# tickers = ['aapl','adbe','amd','fi', 'csco', 'ibm', 'intc', 'lrcx', 'msft', 'mu', 'orcl', 'qcom', 'txn', 'nvda', 'fis', 'crm', 'avgo', 'now']

optimal = optimal_sharpe(ticker_lst, "2020-01-01", "2022-01-01", 0, 1000000)

print(optimal.fun)

[*********************100%***********************]  41 of 41 completed

4 Failed downloads:
['CELG', 'AGN', 'MON', 'RTN']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
/var/folders/9g/h_n63rvn21s1k0y3xvf0n28h0000gn/T/ipykernel_61490/2817871324.py:8: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = data.pct_change()


nan


In [7]:
# Function to get the total volume for a call or put of a given stock.
# ticker: yfinance Ticker class
# put: Boolean for if you want to calculate put volume. Else, put False for call volume. 
def get_options_vol(ticker, put):
    exps = ticker.options # Expiration dates of available options
    optdata = pd.DataFrame() # Data storage
    for exp in exps:
        chain = pd.DataFrame()
        if put: chain = ticker.option_chain(exp).puts['volume'] # Gets the desired columns
        else: chain = ticker.option_chain(exp).calls['volume'] # If put options are desired then use this data.
        optdata = pd.concat([optdata, chain]) # Add the calls/puts to the main dataframe. 
    return optdata.sum()['volume'] # output total volue of put/call options

# Function to calculate the PCR for each stock. 
def PCR_calc(tickers):
    pcrdata = pd.DataFrame(columns=['Ticker', 'Put Volume', 'Call Volume', 'PCR'])
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        try: 
            # Get the volume for Put and Call options:
            call_options = get_options_vol(stock, False)
            put_options = get_options_vol(stock, True)
            # Calculate PCR Ratio:
            pcr = put_options / call_options
            #print(f"Ticker: {ticker}, PCR: {pcr}")  # Debugging
            pcrdata.loc[len(pcrdata)] = [ticker, put_options, call_options, pcr]
        except Exception as e:
            print(f"Error processing {ticker}: {e}")  # Debugging (output error)
            pass
    return pcrdata

In [8]:
# Loading data into variables
stock_filter = filter_stocks(ticker_lst)
ticker_data = stock_filter[0]
ticker_lst = list(ticker_data.keys()) # Reassign original ticker list

$AGN: possibly delisted; no timezone found
$CELG: possibly delisted; no timezone found
$MON: possibly delisted; no timezone found
$RTN: possibly delisted; no timezone found


In [9]:
data = pd.DataFrame()
for ticker in ticker_data:
    data[ticker] = ticker_data[ticker]

# returns = data.pct_change()
# returns.drop(index=returns.index[0], inplace = True)

data.head()

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BB.TO,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2022-09-30,191.404776,172.704627,129.770337,347.895511,63.618708,158.411311,183.464412,169.738424,40.027893,6.51,...,151.100543,113.588997,37.189999,24.323668,76.140358,202.933840,687.262538,260.523561,208.383263,51.127950
2022-10-03,197.290979,177.993480,133.471946,358.158002,65.896549,162.448693,190.413461,176.705719,41.207520,6.53,...,156.771158,114.575363,37.799999,25.033072,77.623260,209.568080,701.510291,267.798221,210.382686,52.713016
2022-10-04,202.346169,182.716096,138.246467,370.894770,69.286510,169.752435,197.811375,187.163658,42.917320,6.74,...,163.658816,117.543633,42.610001,25.316830,79.079201,216.530071,711.933919,274.644980,217.864602,54.057158
2022-10-05,202.761643,184.440454,138.648807,370.935327,68.737150,169.556174,196.329055,185.201048,42.307625,6.67,...,167.029081,117.041313,41.959999,25.104012,78.620857,220.004520,717.241200,268.279665,216.626223,53.511895
2022-10-06,201.418235,180.528498,137.401540,364.350628,68.013600,168.644967,193.622899,185.327211,41.697927,6.64,...,167.550676,113.762512,41.320000,24.421211,75.751671,218.352485,706.436259,265.056836,213.620574,52.091671


In [ ]:
# Load the PCR values for each of the valid stocks into a variable
options_data = PCR_calc(ticker_lst)

Error processing BB.TO: 'volume'
Error processing RY.TO: 'volume'
Error processing SHOP.TO: 'volume'
Error processing T.TO: 'volume'
Error processing TD.TO: 'volume'


pandas.core.frame.DataFrame

In [11]:
options_data = options_data.sort_values(by='PCR', ascending=True)
options_data['Rank'] = [i for i in range(len(options_data))]
options_data = options_data.set_index('Rank')

# Display the table of rankings based off PCR. 
# The rankings are based off the stocks with the greatest sentiment for if they will go up or not
# The tickers at the top of the list have a high call rate (meaning the price will go up)
options_data

,Ticker,Put Volume,Call Volume,PCR
Rank,,,,
0,UPS,6222.0,28371.0,0.219308
1,MO,6332.0,24459.0,0.258882
2,PM,3768.0,14488.0,0.260077
3,KO,12061.0,43651.0,0.276305
4,BA,46348.0,113708.0,0.407605
5,AMZN,115919.0,279247.0,0.415113
6,PG,4655.0,11025.0,0.422222
7,PEP,6221.0,12720.0,0.489072
8,QCOM,15364.0,27593.0,0.556808


In [12]:
# Define to get call and put option data (specifically the total volume)
# def get_options_vol(ticker, put):
#     exps = ticker.options # Expiration dates of available options
#     data = pd.DataFrame() # Data storage
#     for exp in exps:
#         chain = pd.DataFrame()
#         if put: chain = ticker.option_chain(exp).puts['volume'] # Gets the desired columns
#         else: chain = ticker.option_chain(exp).calls['volume'] # If put options are desired then use this data.
#         data = pd.concat([data, chain]) # Add the calls/puts to the main dataframe. 
#     return data.sum()['volume'] # output total volue of put/call options

#cols = ['lastTradeDate','strike', 'bid', 'ask', 'volume', 'inTheMoney', 'currency']
# chain = chain.set_index('lastTradeDate') # Reset the index to the expiration dates
# chain.index = chain.index.strftime('%Y-%m-%d') # Remove excess data
# chain = chain.rename_axis('Expirations') # Rename the index 

In [13]:
# Code to output final dataframe to a CSV file called Stocks_Group_XX.csv
Stocks_Final = Portfolio_Final[['Ticker', 'Shares']]
Stocks_Final.to_csv(f'Stocks_Group_{group}.csv', index=False)

In [14]:
Portfolio_Final

,Ticker,Price,Currency,Shares,Value,Weight


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

---
<p style="color: #004dd3">
Akram Jamil
</p>

<p style="color: #2C8CA9">
Jester Yang
</p>

<p style="color: #3cc19d;">
Annie Wong
</p>

---